In [105]:
from solid import *
from solid.utils import *

from subprocess import run

import viewscad
r = viewscad.Renderer()

In [106]:
import os, solid; print(os.path.dirname(solid.__file__) + '/examples')

/home/flo/.local/lib/python3.9/site-packages/solid/examples


In [107]:
d = cube([7,4,4]) + sphere(5) - cylinder(r=2, h=6)
r.render(d)

In [108]:
# CONFIGURATION
GENERATE_WITH_KEYCAPS = True

In [109]:
KEYCAP_SIZE = 18
KEYCAP_HEIGHT = 9
KEYCAP_Z_OFFSET = 6.6

PLATE_GRID = 19.05
PLATE_HOLE = 14
PLATE_THICKNESS = 4
# PLATE_SPACER = SWITCH_GRID - PLATE_HOLE  # = 5.05mm

def plate_cutout(units_x=1, units_y=1, withCap=False):
    offset = units_x - 1;
    plate = cube([PLATE_GRID*units_x,PLATE_GRID*units_y,PLATE_THICKNESS], center=True)
    hole = cube([PLATE_HOLE,PLATE_HOLE,PLATE_THICKNESS+0.2], center=True)
    cutout = plate - hole
    
    # TODO: if units_x or units_y >= 2: add stab holes

    if(withCap):
        cap = color([0.0,1.0,0.5])(cube([KEYCAP_SIZE*units_x,KEYCAP_SIZE*units_y,KEYCAP_HEIGHT], center=True))
        cap = up(PLATE_THICKNESS/2 + KEYCAP_HEIGHT/2 + KEYCAP_Z_OFFSET)(cap)
        cutout += cap
    
    return cutout

r.render(plate_cutout(units_x=1, units_y=2, withCap=True))

In [110]:
# keys[units_y], units_x
def plate_column(keys=[1,1,1,1], units_x=1):
    i=0
    offset=0
    column = cube(0)
    while(i<len(keys)):
        units_y = keys[i]
        cutout = plate_cutout(units_x=units_x, units_y=units_y, withCap=GENERATE_WITH_KEYCAPS)
        cur_offset = (units_y-1)/2
        column += forward(PLATE_GRID*(offset + cur_offset))(cutout)
        offset += ((units_y+1)/2) + cur_offset
        i += 1
    
    return column
        
r.render(plate_column(keys=[1,1,2,1], units_x=1))

In [115]:
# [column[keys[],units,y_offset_units]]
# keep the y_offset_units in multiple or halfs of 1u -> for easier alignment in KiCAD later
C4 = [1,1,1,1]
C5 = [1,1,1,1,1]
C3_vert = [1.5,1.5,1]

layout = [[C4,1.5,0],[C4,1,0],[C5,1,-0.5],[C5,1,-0.5+0.125],[C5,1,-0.5],[C4,1,0.25],[C3_vert,1,0.25]]

i = 0
offset = 0
plate = cube(0)
while(i<len(layout)):
    column_specs = layout[i]
    column = plate_column(keys=column_specs[0],units_x=column_specs[1])
    cur_offset = (column_specs[1]-1)/2
    plate += right(PLATE_GRID*(offset+cur_offset))(forward(PLATE_GRID*column_specs[2])(column))
    offset += ((column_specs[1]+1)/2) + cur_offset
    i += 1

    
r.render(plate)

In [112]:
projection(d) # muss ich vlt doch in openscad machen

In [113]:
# generate valid openscad code and store it in file
scad_render_to_file(plate, 'plate.scad')

# run openscad and export to stl
run(["openscad", "-o",  "plate.stl", "plate.scad"])

CompletedProcess(args=['openscad', '-o', 'plate.stl', 'plate.scad'], returncode=0)